In [ ]:
import numpy as np 
import matplotlib.pyplot as plt 
import ipywidgets
import ipywidgets as widgets
from scipy.signal import detrend, savgol_filter, butter,sosfiltfilt
from scipy.optimize import curve_fit
from scipy.interpolate import interp1d
import os     
import pandas as pd
import json

In [ ]:
#Setting plots parameters
%matplotlib inline
plt.rcParams['figure.dpi'] = 300
plt.rcParams['font.size']= 15
plt.style.use("seaborn-colorblind")

# Introduction
This notebook allows to analyse stress relaxation data obtained from the Chiaro/Piuma/ nanoindenters from Optics 11 Life. It is divided into several sections that should be ran in order. The notebook runs ipywidgets to make it more user friendly for non-programmers.
This is a work in progress and the notebook is at a very early stage. Please feel free to report any bugs and improvements!


## Functions
This is the "backend" and needs only to be ran, but not modified.

In [ ]:
def get_files(dir_path,ext=".txt"): #edit to be JSON so that cleaned file from nanopreapre can be read! 
    """Gets files from one directory and stores them into a list."""
    files_list = [] #list of file names
    files_dir = [] #list of file directories
    for root, dirs, files in os.walk(dir_path):
        for name in files:
        #get only txt files but exclude position.txt
            if ext==".txt":
                if name.endswith(ext) and not name.endswith('position.txt'):
                    files_list.append(name)
                    files_dir.append(os.path.join(root, name))
            else:
            #for other extensions, do not exclude files
                if name.endswith(ext):
                    files_list.append(name)
                    files_dir.append(os.path.join(root, name))
    return files_list, files_dir

def read_file(f): 
    """
    Reads one .txt file 
    """
    with open(f, encoding='utf-8', errors='ignore') as dynamic:
        stopLine = 'Time (s)'
        numeric = False
        data = []
        for riga in dynamic:
            if numeric is False:
                if riga[0:len(stopLine)] == stopLine:
                    numeric = True
            else:
                line = riga.strip().replace(',', '.').split('\t')
                # Time (s) Load (uN) Indentation (nm) Cantilever (nm) Piezo (nm) Auxiliary
                # skip  #5 auxiliary if present
                data.append([float(line[0]), float(line[1])*1000.0, float(line[2]),
                            float(line[3]), float(line[4])])
        data = np.array(data)
    return data

def getMedCurve(xar, yar,loose=True, threshold=3, error=False):
    """
    Takes repeated nummerical data (replicates stored in a multi dimensional list) 
    and computes the average and error. Useful for displaying "average" plots
    with error bands.
    This function was taken from the following github repo (https://github.com/CellMechLab/nanoindentation),
    author Prof Massimo Vassalli at the Cellular Mechanobiology Lab, University of Glasgow.
    """
    if loose is False:
        xmin = -np.inf
        xmax = np.inf
        deltax = 0
        nonecount = 0
        for x in xar:
            if x is not None and np.min(x) is not None:
                xmin = np.max([xmin, np.min(x)])
                xmax = np.min([xmax, np.max(x)])
                deltax += ((np.max(x)-np.min(x))/(len(x)-1))
            else:
                nonecount += 1
        deltax /= (len(xar)-nonecount)
        xnew = np.linspace(xmin, xmax, int((xmax-xmin)/(deltax)))
        ynew = np.zeros(len(xnew))
        for i in range(len(xar)):
            if xar[i] is not None and np.min(xar[i]) is not None:
                ycur = np.interp(xnew, xar[i], yar[i])
                ynew += ycur
        ynew /= (len(xar)-nonecount)
    else:
        xmin = np.inf
        xmax = -np.inf
        deltax = 0
        for x in xar:
            try:
                xmin = np.min([xmin, np.min(x)])
                xmax = np.max([xmax, np.max(x)])
                deltax += ((np.max(x) - np.min(x)) / (len(x) - 1))
            except TypeError:
                return
        deltax /= len(xar)
        xnewall = np.linspace(xmin, xmax, int((xmax - xmin) / deltax))
        ynewall = np.zeros(len(xnewall))
        count = np.zeros(len(xnewall))
        ys = np.zeros([len(xnewall), len(xar)])
        for i in range(len(xar)):
            imin = np.argmin((xnewall - np.min(xar[i])) ** 2)  # +1
            imax = np.argmin((xnewall - np.max(xar[i])) ** 2)  # -1
            ycur = np.interp(xnewall[imin:imax], xar[i], yar[i])
            ynewall[imin:imax] += ycur
            count[imin:imax] += 1
            for j in range(imin, imax):
                ys[j][i] = ycur[j-imin]
        cc = count >= threshold
        xnew = xnewall[cc]
        ynew = ynewall[cc] / count[cc]
        yerrs_new = ys[cc]
        yerr = []
        for j in range(len(yerrs_new)):
            squr_sum = 0
            num = 0
            std = 0
            for i in range(0, len(yerrs_new[j])):
                if yerrs_new[j][i] != 0:
                    squr_sum += (yerrs_new[j][i] - ynew[j]) ** 2
                    num += 1
            if num > 0:
                std = np.sqrt(squr_sum / num)
            yerr.append(std)
        yerr = np.asarray(yerr)
    if error == False:
        return xnew[:-1], ynew[:-1]
    elif error == True:
        return xnew[:-1], ynew[:-1], yerr[:-1]

def get_times_DMA(f):
    """
    Returns touple of arrays with start and end times of DMA sweeps
    """
    with open(f, encoding='utf-8', errors='ignore') as dynamic:
        target_start = 'DMA absolute start times (s)'
        target_end = 'DMA absolute end times (s)'
        for riga in dynamic:
            if riga[0:len(target_start)]==target_start:
                good_start = riga[len(target_start):].strip()
                better_start = list(map(float, good_start.split(',')))
            if riga[0:len(target_end)]==target_end:
                good_end = riga[len(target_end):].strip()
                better_end = list(map(float, good_end.split(',')))
    return better_start, better_end

def ft_relax(g, t, g_0=1, g_dot_inf=0, N_f=100, interpolate=True, oversampling=10):
    """ Calculates the Fourier transform of numeric data.

    Takes any numeric time-dependent function g(t) that vanishes fot t<0,
    sampled at finite points [g_k,t_k] with k=1...N, and returns its 
    Fourier transform g(omega), together with the frequency range omega 
    defined from 1/t_max to 1/t_min. For details on the numerical procedure,
    refer to Tassieri et al., 2016 (https://doi.org/10.1122/1.4953443).

    Parameters
    ---------
    g : array 
        measured time-dependent variable.
    t : array 
        time array. 
    g_0: 
        value of g at time euqal 0. Can be taken as g[0].
    g_dot_inf:
        value of the time derivative of g at time equal infinity. Can be taken as 0.
    N_f: int
        frequency samples.
    interpolate: bool
        if True, data is interpolated with a cubic spline and re-sampled in log-space.
    oversampling: int
        factor by which the length of the time array is increased for oversampling in log-space.
    """
    g = np.array(g)
    t = np.array(t)

    if interpolate is True:
        gi = interp1d(t, g, kind='cubic', fill_value='extrapolate')
        t_new = np.logspace(min(np.log10(t)), max(np.log10(t)), len(
            t)*oversampling)  # re-sample t in log space
        g = gi(t_new)  # get new g(t) taken at log-space sampled t
        t = t_new
    i = complex(0, 1)
    min_omega = 1/max(t)
    max_omega = 1/min(t)
    N_t = len(t)
    omega = np.logspace(np.log10(min_omega), np.log10(max_omega), N_f)
    zero = i*omega*g_0 + (1-np.exp(-i*omega*t[1]))*((g[1]-g_0)/t[1])\
        + g_dot_inf*np.exp(-i*omega*t[N_t-1])
    res = np.zeros(len(omega), dtype=complex)
    for w_i, w in enumerate(omega):
        after = 0
        for k in range(2, N_t):
            after += ((g[k] - g[k-1]) / (t[k] - t[k-1])) * (np.exp(-i * w *
                                                                   t[k-1])-np.exp(-i * w * t[k]))
        res[w_i] = (zero[w_i]+after)
    return omega, ((res)/(i*omega)**2)  # is omega Hz or rad/s

def linear(x,a,b):
    return a*x +b

def sls_model(x,E_1,E_2,tau):
    "standard linear solid model (prony)."
    E_t = E_2*np.exp(-x/tau)  #Relaxation modulus  
    return ((4/3 * np.sqrt(R)) * ((E_1+E_t)/(1-nu**2)) * delta_0**(3/2))

def prony_model(x,E_1,E_2,E_3,tau2,tau3):
    "standard linear solid model (prony)."
    E_t = E_2*np.exp(-x/tau2) + E_3*np.exp(-x/tau3) #Relaxation modulus (Prony Series) 
    return ((4/3 * np.sqrt(R)) * ((E_1+E_t)/(1-nu**2)) * delta_0**(3/2))

def do_fit(model,xdata,ydata,guess=None):
    """Fits data based on a given model."""
    xdata=np.asarray(xdata)
    ydata=np.asarray(ydata)
    popt, pcov = curve_fit(model,xdata,ydata,maxfev=10000,p0=guess)
    xdata=np.linspace(min(xdata),max(xdata),1000)
    return [xdata, model(xdata,*popt)]

def linear_detrend(tdata,fdata):
    #Isolate drifted signal
    start = np.argmin((tdata-10.0)**2) #end of fast relaxation
    end = np.argmin((tdata-55.0)**2) #end of signal 
    fdata_drift = fdata[start:end]
    tdata_drift = tdata[start:end]
    popt,pcov=curve_fit(linear,tdata_drift,fdata_drift)
    time_all = np.linspace(min(tdata),max(tdata),len(tdata))
    fmodel = linear(time_all,*popt)
    de_drifted = fdata-fmodel
    return de_drifted,popt

## Input Data 
Below, input the directory to the cleaned JSON file originating from NanoPrepare. 

In [ ]:
dirw=widgets.Text(
    value='',
    placeholder='Please enter the files directory',
    description='Directory:',
    disabled=False
)
display(dirw)

## Screening 
The plot below serves to *select* the thresholds for screening and aligning curves. This section is used only to selct the threshold; the actual data is thresholded in the next section (**Adjusting**) if the "Threshold" checkbox is checked. The data can be thresholded based on the following thresholds: 

1. tmin (s): the time under which the maximum force should occur. Any curve whose maximum force occurs after this time is discarded from the analysis. 
2. tmax (s): the maximum time one wants to display and analyse the data for. This is used to essentially remove the ramping down of the stress relaxation curve. Alla data points after this time are discarded.
3. fmin (uN): the minimum acceptable value for the maximum (peak) force. Any curve whose maximum force is smaller than fmin will be discarded.
4. fmax (uN): the maximum acceptable value for the maximum (peak) force. Any curve whose maximum force is greater than fmax will be discarded.

tip: adjust the sliders by typing the approximate value for the variable in the box next to the slider and click enter on your keyboard. Just sliding will update the plot for each value the slider passes through, resulting in a laggy visual update.

In [ ]:
f = open(dirw.value)
data = json.load(f)
def first_plot(tmin=0.5,tmax=30.0,fmin=0.0,fmax=50.0):
    fig,ax=plt.subplots(1,2,figsize=(8,3))
    tall = []
    fall = []
    for i in range(len(data['curves'])):
        raw_time = np.array(data['curves'][i]['raw_data']['raw_time'])
        raw_force = np.array(data['curves'][i]['raw_data']['raw_force'])
        for_force = np.array(data['curves'][i]['data']['F']) #for = forward segment
        for_Z = np.array(data['curves'][i]['data']['Z'])
        
        #Offset force-time curves 
        off = raw_force[0] #first point
        #if forst point is negative, align
        if off <0: 
            raw_force = raw_force-off
        else:
            raw_force = raw_force
        
        #Offset F-z forward curves
        for_off = for_force[0]
        
        if for_off <0: 
            for_force = for_force-for_off
        else:
            for_force= for_force
    
        ax[0].plot(raw_time,raw_force,alpha=1,lw=0.1,c='k')
        ax[1].plot(for_Z,for_force, lw=0.1,c='k')
        ax[0].axhline(fmin,c="salmon",lw=1)
        ax[0].axhline(fmax,c="salmon",lw=1)
        ax[0].axvline(tmax,lw=1)
        ax[0].axvline(tmin,lw=1)
    ax[0].set_xlabel('time (s)')
    ax[0].set_ylabel('force (uN)')
    ax[1].set_ylabel('force (N)')
    ax[1].set_xlabel('Z (m)')
    fig.tight_layout()
    plt.show()
plot1w=widgets.interactive(first_plot,tmin=(0.0,50.0,0.5), tmax=(0.0,100.0,0.5),fmin=(0.0,100.0,0.1),fmax=(0.0,1500.0,0.1))
display(plot1w)

## Adjusting
Now click the "Threshold" checkbox if you want to apply the above-specified thresholds to the data.

Given the above specified thresholds, the code below finds the maximum force and corresponding time and aligns this value to 0. All curves should now start from t=0. Note that the maximum time displayed will be different from the one selected above as curves are shifted by the time at which the maximum force occurs (however relative time interval is preserved)!

In the end, the average curve is plotted (red) on top of the individual curves (black).

In [ ]:
#Setting plots parameters
%matplotlib qt
plt.rcParams['figure.dpi'] = 300
plt.rcParams['font.size']= 8
plt.style.use("seaborn-colorblind")

In [ ]:
thresholdw=widgets.Checkbox(
    value=False,
    description='Threshold data',
    disabled=False
)

display(thresholdw)

In [ ]:
#Widget parameters (Global thresholds)
t_min=plot1w.kwargs["tmin"] #s #time under which max force should occur
t_max=plot1w.kwargs["tmax"] #s #max time to display and analyse data for
f_min=plot1w.kwargs["fmin"] #The minium acceptable value for the max force (uN)
f_max = plot1w.kwargs["fmax"]#The maximum acceptable value for the max force (uN)

tall = []
fall = []
fnormall=[]
fig,ax = plt.subplots(1,1,figsize=(4,2))
for i in range(len(data['curves'])):
    raw_time = np.array(data['curves'][i]['raw_data']['raw_time'])
    raw_force = np.array(data['curves'][i]['raw_data']['raw_force'])
    for_force = np.array(data['curves'][i]['data']['F']) #for = forward segment
    for_Z = np.array(data['curves'][i]['data']['Z'])
    #Offset force-time curves 
    off = raw_force[0] #first point
    #if forst point is negative, align
    if off <0: 
        raw_force = raw_force-off
    else:
        raw_force = raw_force
    #Clean data based on user-selected thresholds
    #NB: to be applied on raw data 
    fmax = max(raw_force)
    imaxf = np.argmin((raw_force-fmax)**2)
    if thresholdw.value is True:
        if (fmax > f_max) or (fmax < f_min) or raw_time[imaxf] > t_min:
            continue 
    #Align data to 0 and slice to user-selcted max time
    itmax =np.argmin((raw_time-t_max)**2)
    t=raw_time[imaxf:itmax]-raw_time[imaxf]
    f=raw_force[imaxf:itmax]
    fnorm=f/max(f) #normalised force
    tall.append(t)
    fall.append(f)
    fnormall.append(fnorm)
ax.set_title("Average curve")
ax.set_xlabel('Time (s)')
ax.set_ylabel('Force (uN)')

#Find and plot average curve
t_av,f_av,f_err=getMedCurve(tall,fall,error=True) #average curve
_,_,f_err_norm=getMedCurve(tall,fnormall,error=True) #average normalised curve
discard = 30
t_av = t_av[:-discard]
f_av = f_av[:-discard]
f_err = f_err[:-discard]
f_err_norm=f_err_norm[:-discard]
ax.errorbar(t_av,f_av,c='tomato',lw=1,ls='-',alpha=1) 
fig.tight_layout()
plt.show()

If the average curve has an upwards trend (temperature drift), detrend data:

In [ ]:
detrendw=widgets.Checkbox(
    value=False,
    description='Detrend data',
    disabled=False
)

display(detrendw)

In [ ]:
fig,axs = plt.subplots(1,2,figsize=(6,1.5))
if detrendw.value is True: 
    #average curve (absolute)
    f_av_detrended,optd = linear_detrend(t_av,f_av) 
    f_av_detrended = f_av_detrended+optd[1] #add back y intercept for correct f scaling
    f_av = f_av_detrended
    #error (absolute)
    f_err_detrended,optd = linear_detrend(t_av,f_err) 
    f_err_detrended = f_err_detrended+optd[1]
    f_err=f_err_detrended
    #error (from normalised curves)
    f_err_detrended_norm,optd = linear_detrend(t_av,f_err_norm) 
    f_err_detrended_norm = f_err_detrended_norm+optd[1]
    f_err_norm=f_err_detrended_norm
    
axs[0].plot(t_av,f_av)
axs[1].plot(t_av,f_av/max(f_av))
axs[0].set_title("Data")
axs[1].set_title("Normalised Data")
axs[0].set_xlabel("Time (s)")
axs[1].set_xlabel("Time (s)")
axs[0].set_ylabel("Force (uN)")
axs[1].set_ylabel("Norm. Force")
fig.tight_layout()

## Fitting data
Fit average curve with relaxation model. This can be a Standard linear solid (SLS) or Prony series; i.e. a generalised maxwell model with two relaxation times, see for example https://pubs.rsc.org/en/content/articlelanding/2020/sm/c9sm01020c or https://pubs.rsc.org/en/content/articlelanding/2020/bm/c9bm01339c).
The SLS model has 3 fitting parameters ($E_1$, the long term elastic modulus - $E_2$, the relaxation modulus - and its associated time constant, $\tau_2$).
The prony series model has 4 fitting parameters ($E_1$, the long term elastic modulus - $E_2$ and $E_3$, the relaxation moduli - and their associated time constants, $\tau_2$ and $\tau_3$).
The user needs to enter the Poisson's ratio, the tip radius and the approximate constant indentation depth; as Hertzian contact is still assumed. After, the user is prompted with what model to fit the average curve.

In [ ]:
poisson = widgets.BoundedFloatText(
    value=0.5,
    min=0,
    max=1.0,
    step=0.1,
    description='Poisson\'s:',
    disabled=False
)
display(poisson)

radius = widgets.BoundedFloatText(
    value=3,
    min=1,
    max=250,
    step=0.5,
    description='$R$ (um):',
    disabled=False
)
display(radius)

ind_0 = widgets.BoundedFloatText(
    value=3,
    min=1,
    max=250,
    step=0.5,
    description='$\delta_0$ (um):',
    disabled=False
)
display(ind_0)

chosen_model = widgets.Dropdown(
    options=['SLS', 'Prony'],
    value='Prony',
    description='Model:',
    disabled=False,
)
display(chosen_model)

In [ ]:
nu = poisson.value
R = radius.value
delta_0 = ind_0.value
fig,ax=plt.subplots(1,1,figsize=(4,2))
if chosen_model.value=="Prony":
    model_name = "Prony"
    seed = [f_av[-1],(f_av[-1]+f_av[0]),(f_av[-1]+f_av[0]),0.1,10.0]
    popt,pcov = curve_fit(prony_model,t_av,f_av,sigma=f_err,p0=seed,maxfev=100000) #method='trf', #loss="soft_l1"
    perr = np.sqrt(np.diag(pcov))
    ax.plot(t_av,f_av,lw=4,label="Raw data")
    ax.plot(t_av,prony_model(t_av,*popt),'--',c='r',lw=2,label=model_name +" Model")
    norm_model=prony_model(t_av,*popt)/max(f_av)
if chosen_model.value=="SLS":
    model_name = "SLS"
    seed = [f_av[-1],(f_av[-1]+f_av[0]),10.0]
    popt,pcov = curve_fit(sls_model,t_av,f_av,sigma=f_err,p0=seed,maxfev=100000) #method='trf', #loss="soft_l1"s
    perr = np.sqrt(np.diag(pcov))
    ax.plot(t_av,f_av,lw=4,label="Raw data")
    ax.plot(t_av,sls_model(t_av,*popt),'--',c='r',lw=2,label=model_name+" Model")
    norm_model=sls_model(t_av,*popt)/max(f_av)
plt.legend()
ax.set_xlabel("Time (s)")
ax.set_ylabel("Force (uN)")
fig.tight_layout()

Final plot to save for representative purposes:

In [ ]:
fig,ax=plt.subplots(1,1,figsize=(3.5,2))
f_av_norm=f_av/max(f_av)
ax.plot(t_av,f_av_norm,label="Average Data")
ax.fill_between(t_av,f_av_norm-0.5*f_err_norm,f_av_norm+0.5*f_err_norm,alpha=0.3,label="1SD")
ax.plot(t_av,norm_model,'--',c='r',lw=1,label=model_name+" Model")
ax.set_xlabel("Time (s)")
ax.set_ylabel("Norm. Force")
plt.legend()
fig.tight_layout()

 ## Saving data
 The cell below saves the data from the average curve in a .tsv file, together with the fitted model and best model parameters:

In [ ]:
savingfolderw=widgets.Text(
    placeholder='Output folder',
    disabled=False
)
boxw1=widgets.HBox([widgets.Label(value="Output Folder"), savingfolderw])
display(boxw1)

samplenamew=widgets.Text(
    placeholder='Please enter the sample name',
    disabled=False
)
boxw2=widgets.HBox([widgets.Label(value="Sample Name"), samplenamew])
display(boxw2)

In [ ]:
fname=savingfolderw.value+"/"+samplenamew.value +".tsv"
with open(fname,"w") as f: 
    f.write("Exported stress relaxation analysis \n")
    f.write("Sample Name: {} \n".format(samplenamew.value))
    f.write("Fitted model: {} \n".format(model_name))
    if model_name == "Prony":
        f.write("Model tau_2 (s) {} pm {} \n".format(popt[3],perr[3]))
        f.write("Model tau_3 (s) {} pm {} \n".format(popt[4],perr[4]))
    if model_name =="SLS":
        f.write("Model tau_1 (s) {} pm {} \n".format(popt[2],perr[2]))
    f.write('Avg time [s] \t Avg Norm Force \t Error Force \t Normalised Model \n')
    for x in zip(*[t_av,f_av_norm,f_err_norm,norm_model]):
                 f.write("{0}\t{1}\t{2}\t{3}\n".format(*x))

## Relaxation time
Below, you can enter the stress value for which you wish to calculate the relaxation time. For example, 0.5 means calculating the time for which the stress relaxes to half of its original value. The average curve is used for this calculation as single curves are too noisy.

In [ ]:
relaxation_timew=widgets.BoundedFloatText(
    value=0.5,
    min=0,
    max=1.0,
    step=0.1,
    disabled=False
)
boxw=widgets.HBox([widgets.Label(value="Enter stress value for which relaxation time is calculated:"), relaxation_timew])
display(boxw)

In [ ]:
#Extract time at which force reaches % of original value (specified above)
F_TARGET=relaxation_timew.value*(max(f_av_norm)) 
i_fclose = np.argmin((f_av_norm-F_TARGET)**2) 
t_target=t_av[i_fclose]
print(f"The time for which the stress relaxes to {(relaxation_timew.value*100.0)}% of the original value is {t_target} s!")

def plot_average(): 
    plt.plot(t_av,f_av_norm,zorder=-1)
    plt.scatter(t_target,f_av_norm[i_fclose],c='red',zorder=1,alpha=0.5)
    plt.xlabel('Time (s)')
    plt.ylabel('Normalised force')
    plt.show()
    
widgets.interactive(plot_average)

# Fitting single curves with given model

Below, single $F-t$ curves are fitted using a stress relaxation model (Standard linear solid (SLS) or Prony series; i.e. a generalised maxwell model with two relaxation times, see for example https://pubs.rsc.org/en/content/articlelanding/2020/sm/c9sm01020c or https://pubs.rsc.org/en/content/articlelanding/2020/bm/c9bm01339c).
The SLS model has 3 fitting parameters ($E_1$, the long term elastic modulus - $E_2$, the relaxation modulus - and its associated time constant, $\tau_2$).
The prony series model has 4 fitting parameters ($E_1$, the long term elastic modulus - $E_2$ and $E_3$, the relaxation moduli - and their associated time constants, $\tau_2$ and $\tau_3$).
The user needs to enter the Poisson's ratio, the tip radius and the approximate constant indentation depth; as Hertzian contact is still assumed. After, the user is prompted with what model to fit the single curves.

In [ ]:
# poisson = widgets.BoundedFloatText(
#     value=0.5,
#     min=0,
#     max=1.0,
#     step=0.1,
#     description='Poisson\'s:',
#     disabled=False
# )
# display(poisson)

# radius = widgets.BoundedFloatText(
#     value=3,
#     min=1,
#     max=250,
#     step=0.5,
#     description='$R$ (um):',
#     disabled=False
# )
# display(radius)

# ind_0 = widgets.BoundedFloatText(
#     value=3,
#     min=1,
#     max=250,
#     step=0.5,
#     description='$\delta_0$ (um):',
#     disabled=False
# )
# display(ind_0)

# chosen_model = widgets.Dropdown(
#     options=['SLS', 'Prony'],
#     value='Prony',
#     description='Model:',
#     disabled=False,
# )
# display(chosen_model)

In [ ]:
# #Widget parameters (Global thresholds)
# t_min=plot1w.kwargs["tmin"] #s #time under which max force should occur
# t_max=plot1w.kwargs["tmax"] #s #max time to display and analyse data for
# f_min=plot1w.kwargs["fmin"] #The minium acceptable value for the max force (uN)
# f_max = plot1w.kwargs["fmax"]#The maximum acceptable value for the max force (uN)

# #Constants for fitting procedure
# nu = poisson.value
# R = radius.value
# delta_0 = ind_0.value
# tau2_all = [] # to append in for loo
# tau3_all = [] # to append in for loop
# fig,ax = plt.subplots(1,1,figsize=(4,2))
# for i in range(len(data['curves'])):
#     raw_time = np.array(data['curves'][i]['raw_data']['raw_time'])
#     raw_force = np.array(data['curves'][i]['raw_data']['raw_force']) 
#     for_force = np.array(data['curves'][i]['data']['F']) #for = forward segment
#     for_Z = np.array(data['curves'][i]['data']['Z'])
#     #Offset force-time curves 
#     off = raw_force[0] #first point
#     #if forst point is negative, align
#     if off <0: 
#         raw_force = raw_force-off
#     else:
#         raw_force = raw_force
#     #Clean data based on user-selected thresholds
#     #NB: to be applied on raw data 
#     fmax = max(raw_force)
#     imaxf = np.argmin((raw_force-fmax)**2)
        
#     if thresholdw.value is True:
#         if (fmax > f_max) or (fmax < f_min) or raw_time[imaxf] > t_min:
#             continue   
#     #Align data to 0 and slice to user-selcted max time
#     itmax =np.argmin((raw_time-t_max)**2)
#     t=raw_time[imaxf:itmax]-raw_time[imaxf] #s
#     t_mod = np.linspace(min(t),max(t),1000)
#     f=raw_force[imaxf:itmax] #uN
#     #Smooth data with Sav-gol filter 
#     win = 101
#     f=savgol_filter(f,win,3)
#     f=f[win:-win]
#     t=t[win:-win]
    
#     #Filter low-frequency noise
#     # fs = 1/t[1]-t[0]
#     # filt = butter(5, 0.0005, fs=fs,btype='lowpass', output='sos')
#     # f = sosfiltfilt(filt, f)
    
#     if chosen_model.value=="Prony":
#         try:
#             seed = [f[-1],(f[-1]+f[0]),(f[-1]+f[0]),0.1,10.0]
#             popt,pcov = curve_fit(prony_model,t,f,p0=seed,maxfev=100000) #method='trf', #loss="soft_l1"
#             residuals = f - prony_model(t,*popt)
#             plt.plot(t_mod,prony_model(t_mod,*popt),'--',c='r',lw=0.5)
#         except:
#             continue
    
#     if chosen_model.value=="SLS":
#         try:
#             seed = [f[-1],(f[-1]+f[0]),10.0]
#             popt,pcov = curve_fit(sls_model,t,f,p0=seed,maxfev=100000) #method='trf', #loss="soft_l1"s
#             residuals = f - sls_model(t,*popt)
#             plt.plot(t_mod,sls_model(t_mod,*popt),'--',c='r',lw=0.1)
#         except:
#             continue
            
#     ss_res = np.sum(residuals**2)
#     ss_tot = np.sum((f-np.mean(f))**2)
#     R_squared = 1 - (ss_res / ss_tot)
#     if R_squared > 0.8: #get fits where R**2 > 0.8
#         tau2_all.append(popt[3])
#         tau3_all.append(popt[4])
#     plt.plot(t,f,'-',c="k",alpha=0.5,lw=1)
#     ax.set_xlabel("Time (s)")
#     ax.set_ylabel("Force (uN)")
#     plt.show()

In [ ]:
# savingfolderw=widgets.Text(
#     placeholder='Output folder',
#     disabled=False
# )
# boxw1=widgets.HBox([widgets.Label(value="Output Folder"), savingfolderw])
# display(boxw1)

# samplenamew=widgets.Text(
#     placeholder='Please enter the sample name',
#     disabled=False
# )
# boxw2=widgets.HBox([widgets.Label(value="Sample Name"), samplenamew])
# display(boxw2)

In [ ]:
# sample_name=samplenamew.value
# sample_name = sample_name + ".csv"
# data={"tau_2 (s)": tau2_all, "tau_3 (s)": tau3_all}
# df=pd.DataFrame(data)
# df.to_csv(os.path.join(savingfolderw.value,sample_name),index=False)